In [1]:
import Pkg; 

cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMolNotebooks
using MorphoMol
using PyCall
using JLD2
using LinearAlgebra
using NearestNeighbors
using Rotations
using GLMakie

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [2]:
function get_persistence_diagram(points)
    py"""
    import oineus as oin
    import numpy as np
    import torch
    import diode

    def get_persistence_diagram(points):
        points = np.asarray(points)
        simplices = diode.fill_alpha_shapes(points)
        fil = oin.Filtration_double([oin.Simplex_double(s[0], s[1]) for s in simplices])

        dcmp = oin.Decomposition(fil, True)
        params = oin.ReductionParams()
        dcmp.reduce(params)
        dgm = dcmp.diagram(fil, include_inf_points=False)
        return dgm
    """
    py"get_persistence_diagram"(points)
end

get_persistence_diagram (generic function with 1 method)

In [3]:
function get_tps(dgms)
    [MorphoMol.Energies.get_total_persistence(dgm) for dgm in dgms]
end
    
function get_fsols(folder)
    files = readdir(folder)
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[6]) for i in 1:n]
end

function get_thetas(folder)
    files = readdir(folder)
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[7]) for i in 1:n]
end

function txt_to_points(input_file)
    [[parse(Float64, s) for s in split(p," ")] for p in readlines(input_file)[3:end]]
end

txt_to_points (generic function with 1 method)

In [4]:
function get_configurations_from_folder(folder)
    files = readdir(folder)
    realizations = [Vector{Vector{Vector{Float64}}}([]) for i in 1:length(files)]
    for file in readdir(folder)
        centers = txt_to_points("$(folder)$(file)")
        n_atoms_per_mol = Int(length(centers) / 2)
        c1 = centers[1:n_atoms_per_mol]
        c2 = centers[n_atoms_per_mol+1:end]
        realizations[parse(Int, split(file, ".")[1]) + 1] = [c1, c2]
    end
    realizations
end

get_configurations_from_folder (generic function with 1 method)

In [5]:
function folder_of_txts_to_jld2_with_persistence_computation(folder)
    mol_type = split(folder, "/")[end-2]
    idx = split(folder, "/")[end-1]
    py_dgms = MorphoMol.Energies.get_interface_diagrams(folder)
    dgms = [[dgm[1], dgm[2], dgm[3], dgm[4]] for dgm in py_dgms]
    tps = get_tps(dgms)
    Es = get_fsols(folder)
    thetas = get_thetas(folder)
    realizations = get_configurations_from_folder(folder)
    @save "../../Data/measure_paths/jld2s/$(mol_type)/$(idx).jld2" dgms tps Es thetas realizations
end

folder_of_txts_to_jld2_with_persistence_computation (generic function with 1 method)

In [75]:
# folder = "../../Data/measure_paths/txts/6r7m/"
# for subdir in readdir(folder)
#     if subdir != ".DS_Store"
#         folder_of_txts_to_jld2_with_persistence_computation("$(folder)$(subdir)/")
#     end
# end

# folder = "../../Data/measure_paths/txts/6r7m/1/"
# dgms = get_interface_diagrams(folder)
# xs = [i for i in 1:length(dgms)]
# dgms_selection = [[dgm[2]; dgm[3]] for dgm in dgms]
# tps = get_tps(dgms_selection)
# Es = get_fsols(folder)
# thetas = get_thetas(folder);

In [51]:
# @load "../../Data/collected_simulation_results/rwm_ma_2_6r7m/52.jld2" input output
# realizations = [[e for e in eachcol(hcat(MorphoMol.Utilities.get_matrix_realization(state, input["template_centers"])...))] for state in output["states"]]
# pdgms = [get_persistence_diagram(r) for r in realizations]
# dgms = [[dgm[1], dgm[2], dgm[3]] for dgm in pdgms]
# merge!(output, Dict("PDGMs" => dgms))
# merge!(input, Dict("mol_type" => "6r7m"))
# @save "../../Data/examples/correct_assembly_ma_only.jld2" input output

In [6]:
@load "../../Data/hpc_out/rwm_wp_2_6r7m/8_rwm_wp_2_6r7m.jld2" input output
display(MorphoMolNotebooks.interactive_simulation_analysis(merge!(input, Dict("mol_type" => "6r7m")), output))